# Profiling
### Finding bottlenecks in your Python program

## Contents

* Timing
* Profiling
* Line-by-line profiling

  *"First make it work. Then make it right. Then make it fast."* 
  
  Kent Beck

Prior to code optimization, hotspots and bottlenecks must be located. 

## Profiling allows us to measure resources used by sections of the program. 

Typical resources of interest are
* Amount of CPU used
* Amount of RAM used

But also other resources can be measured, such as:

* Disk I/O
* Network I/O
* ...

Today, we only consider CPU profiling.

## Profiling techniques

Start simple, switch to more complex techniques if needed!

Techniques for measuring CPU time in increasing complexity:
1. Manual timing
2. The `timeit` module
3. The `cprofile` module

## Case study: filling a grid with point values

* Consider a rectangular 2D grid
<center>![xkcd](pdf/grid_lattice.svg "Python")Grid lattice</center>
* A NumPy array `a[i,j]` holds values at the grid points

# An implementation

In [1]:
from numpy import *

class Grid2D(object):
    def __init__(self,
                 xmin=0, xmax=1, dx=0.5,
                 ymin=0, ymax=1, dy=0.5):
        
        self.xcoor = arange(xmin, xmax+dx, step=dx)
        self.ycoor = arange(ymin, ymax+dy, step=dy)

    def gridloop(self, f):
        lx = size(self.xcoor)
        ly = size(self.ycoor)
        a = zeros((lx,ly))

        for i in range(lx):
            x = self.xcoor[i]
            for j in range(ly):
                y = self.ycoor[j]
                a[i,j] = f(x, y)
        return a

# Usage

Create a new grid:

In [2]:
g = Grid2D(dx=0.001, dy=0.001)

Computing grid values:

In [3]:
import numpy
def myfunc(x, y):
    return numpy.sin(x*y) + y

print("Computing values...")
a = g.gridloop(myfunc)
print("done")

Computing values...
done


Getting grid values at a specific point:

In [4]:
i=1000
j=1000
print('value at ({}, {}) is {}'.format(g.xcoor[i], g.ycoor[j], a[i,j]))

value at (1.0, 1.0) is 1.8414709848078965


# Timing

Use `time.time()` to measure the time spend in a code section.
  ```python
  t0 = time.time()
  # execute code here
  t1 = time.time()
  print("Runtime: {}".format(t1-t0))
  ```
  

Timing guidelines:
* Put simple statements in a loop.
* Make sure to use a constant machine load.
* Run the tests several times, choose the **smallest**. Why?

*Note*: 
* time.time() measures **wall clock time**. 
* time.clock() measures **CPU time**.

## Timing of the case study

The case study has two parts that could potentially be slow: 
1. The initialisation `Grid2D(dx=0.001, dy=0.001)`
2. Calling the `g.gridloop(myfunc)` function.

We time these two parts separately to figure out how much time is spend in each.

### Timing the Grid2D initialisation

In [5]:
import time

for i in range(1, 4):
    t0 = time.time()
    g = Grid2D(dx=0.001, dy=0.001)
    t1 = time.time()
    print("Experiment {}, CPU time: {:.4} s".format(i, t1-t0))
print("Done")

Experiment 1, CPU time: 0.0007989 s
Experiment 2, CPU time: 4.268e-05 s
Experiment 3, CPU time: 3.457e-05 s
Done


### Timing the `gridloop` function

In [6]:
import time

for i in range(1, 4):
    t0 = time.time()
    g.gridloop(myfunc)
    t1 = time.time()
    print("Experiment {}. CPU time: {:.4} s".format(i, t1-t0))
print("Done")

Experiment 1. CPU time: 2.065 s
Experiment 2. CPU time: 3.123 s
Experiment 3. CPU time: 2.162 s
Done


$=>$ The gridloop function is causing the slow execution!

## The *timeit* module

The `timeit` module provides an convenient way for measuring the CPU time of small code snippets.

Usage:

In [7]:
import timeit

timeit.timeit(stmt="a+=1", setup="a=0")

0.08905152999795973

The code is automatically wrapped in a for loop. By defaultthe command is executed 1,000,000 times. It returns the **accumulated** runtime:

You can adjust the the number of executions:

In [8]:
timeit.timeit(stmt="a+=1",setup="a=0", number=10000)

0.0007618370000272989

Use timeit.repeat if you would like to repeat the experiment multiple times:

In [9]:
timeit.repeat(stmt="a+=1",setup="a=0", number=10000, repeat=5)

[0.001654998995945789,
 0.0016717429971322417,
 0.0017683210026007146,
 0.0017267280054511502,
 0.0015832589997444302]

## Timing user defined function

Timeit creates its own namespace - which means that variables, functions, ... are not avaible in timeit, unless they are imported in the setup argument:

In [10]:
timeit.repeat(stmt="g.gridloop(myfunc)", setup="from __main__  import g, myfunc", repeat=5, number=1)

[2.8709113790027914,
 2.467202249004913,
 1.9526101250012289,
 1.9533978649997152,
 2.1182920310020563]

## Profiling modules with cProfile

A profile is a set of statistics that describes how often and for how long various parts of the program executed.

`cProfile` is two main (deterministic) profiling module in Python.

### Using cProfile

* As a script: `python -m cProfile script.py
`

* As a module:

In [11]:
import cProfile
pr = cProfile.Profile()
res = pr.run("g.gridloop(myfunc)")  # res contains the statistics
pr.dump_stats("gridloop.prof")  # Dump statistics to file for use with pstats

## Profile a function call while keeping its return value

`pr.run("foo()")` returns an object with the profile statistics. If one needs the return value of `foo()`, use `pr.runcall` instead:

In [12]:
 pr.runcall(g.gridloop, myfunc)

array([[  0.00000000e+00,   1.00000000e-03,   2.00000000e-03, ...,
          9.98000000e-01,   9.99000000e-01,   1.00000000e+00],
       [  0.00000000e+00,   1.00100000e-03,   2.00200000e-03, ...,
          9.98998000e-01,   9.99999000e-01,   1.00100000e+00],
       [  0.00000000e+00,   1.00200000e-03,   2.00400000e-03, ...,
          9.99995999e-01,   1.00099800e+00,   1.00200000e+00],
       ..., 
       [  0.00000000e+00,   1.99799983e-03,   3.99599867e-03, ...,
          1.83730522e+00,   1.83884738e+00,   1.84038870e+00],
       [  0.00000000e+00,   1.99899983e-03,   3.99799867e-03, ...,
          1.83784738e+00,   1.83938924e+00,   1.84093026e+00],
       [  0.00000000e+00,   1.99999983e-03,   3.99999867e-03, ...,
          1.83838870e+00,   1.83993026e+00,   1.84147098e+00]])

## Getting runtime statistics

We can view some runtime statistics with

In [13]:
res.print_stats()

         2004014 function calls in 5.533 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    1.770    0.885    5.533    2.767 <ipython-input-1-bdd39b4c44e8>:11(gridloop)
  2004002    3.761    0.000    3.761    0.000 <ipython-input-3-3e29d12a89c9>:2(myfunc)
        1    0.000    0.000    2.838    2.838 <string>:1(<module>)
        4    0.000    0.000    0.000    0.000 fromnumeric.py:2706(size)
        1    0.000    0.000    2.838    2.838 {built-in method builtins.exec}
        2    0.002    0.001    0.002    0.001 {built-in method numpy.core.multiarray.zeros}
        2    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




Header definitions
* **ncalls**: number of calls
* **tottime**: total time spent in the given function excluding time made in calls to sub-functions
* **percall**: tottime divided by ncalls
* **cumtime**: cumulative time spent in this and all subfunctions
* **percall**: cumtime divided by ncalls
* **filename:lineno(function)**: information about the function

Use the pstats module to get more control over the statistics (sort, restrict, ...).

## The `pstats` module

There are many ways to view profiling data.

The module `pstats` provides the class `Stats` for creating profiling reports:

In [14]:
import pstats
stats = pstats.Stats("gridloop.prof")
stats.sort_stats("time")
stats.print_stats(3)

Thu Sep 28 21:49:52 2017    gridloop.prof

         1002008 function calls in 2.838 seconds

   Ordered by: internal time
   List reduced from 7 to 3 due to restriction <3>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1002001    1.925    0.000    1.925    0.000 <ipython-input-3-3e29d12a89c9>:2(myfunc)
        1    0.912    0.912    2.838    2.838 <ipython-input-1-bdd39b4c44e8>:11(gridloop)
        1    0.001    0.001    0.001    0.001 {built-in method numpy.core.multiarray.zeros}




* The method `sort_stats(key, *keys)` is used to sort future output.
* Common used keys: `'calls', 'cumulative', 'time'`.
* The argument to `print_stats` limits the view to the N most significant results.

## Back to our case-study: What have we learned so far?

The biggest contributors of the total runtime are:
   1. `gridloop` contributes ca 41% of the total runtime.
   2. `myfunc` calls contributes ca 55% of the total runtime.

* `myfunc` is fairly straight-forward
  ```python
  def myfunc(x, y):
    return sin(x*y) + y
  ```
  Might be difficult to improve.
* What about `gridloop`?

## Recall that, `gridloop` was a function of the form

```python
def gridloop(self, f):
    lx = size(self.xcoor)
    ly = size(self.ycoor)
    a = zeros((lx,ly))

    for i in range(lx):
        x = self.xcoor[i]
        for j in range(ly):
            y = self.ycoor[j]
             a[i,j] = f(x, y)
    return a
```

It would be useful to see how much time is spend in each line!

## Line by line profiling

The line_profiler inspects the time spend in each line of a Python function. 

## Usage

1. Install with `pip install line_profiler`
2. "Decorate" the function of interest with `@profile`:
    ```python
    @profile
    def gridloop(func):
        # ...
    ```
3. Run line profiler with:
    ```bash
    kernprof -l -v grid2d_lineprofile.py
    ```

## Demo

In [20]:
!kernprof -l -v grid2d_lineprofile.py

Wrote profile results to grid2d_lineprofile.py.lprof
Timer unit: 1e-06 s

Total time: 4.93353 s
File: grid2d_lineprofile.py
Function: gridloop at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                               @profile
    12                                               def gridloop(self, f):
    13         1            8      8.0      0.0          lx = size(self.xcoor)
    14         1            4      4.0      0.0          ly = size(self.ycoor)
    15         1           18     18.0      0.0          a = zeros((lx,ly))
    16                                           
    17      1002          809      0.8      0.0          for i in range(lx):
    18      1001         1024      1.0      0.0              x = self.xcoor[i]
    19   1003002       759909      0.8     15.4              for j in range(ly):
    20   1002001       982459      1.0     19.9                  y = self.ycoor[j]
    21   1002001      3189301      3.2  

**Conclusion:** A significant amount of time is spend in loops and indexing.

## A vectorised Grid2D implementation

In [16]:
class VectorisedGrid2D(object):
    def __init__(self,
                 xmin=0, xmax=1, dx=0.5,
                 ymin=0, ymax=1, dy=0.5):
        
        self.xcoor = arange(xmin, xmax+dx, step=dx)
        self.ycoor = arange(ymin, ymax+dy, step=dy)

    def gridloop(self, f):
        return f(self.xcoor[:,None], self.ycoor[None,:])  # Vectorized grid evaluation 

## Timing the vectorised Grid2D

In [21]:
vg = VectorisedGrid2D(dx=0.001, dy=0.001)
min(timeit.repeat(stmt="vg.gridloop(myfunc)", setup="from __main__  import vg, myfunc", repeat=5, number=1))

0.03344921799725853

In [22]:
g = Grid2D(dx=0.001, dy=0.001)
min(timeit.repeat(stmt="g.gridloop(myfunc)", setup="from __main__  import g, myfunc", repeat=5, number=1))

1.9516707040020265

**Vectorization yields a 70 times speed improvement!**